## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
##### Module

from keras.datasets import cifar10
#from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
#from keras.models import Model
#from keras.optimizers import Adam
from keras.utils import to_categorical

from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
##### Parameter

#DATASET_PATH  = 'sample'
IMAGE_SIZE = (32, 32)
NUM_CLASSES = 10
BATCH_SIZE = 32
FREEZE_LAYERS = 2
NUM_EPOCHS = 20
WEIGHTS_FINAL = 'model-resnet50-cifer10.h5'

In [4]:
net = ResNet50(include_top=False,
               weights='imagenet',
               input_tensor=None,
               input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3))

C:\Users\Sauce\.conda\envs\megalopolis\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

In [6]:
# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

In [7]:
# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# 輸出整個網路結構
print(net_final.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [9]:
history = net_final.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 173s 3ms/step - loss: 3.0005 - accuracy: 0.2139 - val_loss: 1.9949 - val_accuracy: 0.3742
Epoch 2/20
50000/50000 [==============================] - 159s 3ms/step - loss: 1.8251 - accuracy: 0.3844 - val_loss: 1.4928 - val_accuracy: 0.5319
Epoch 3/20
50000/50000 [==============================] - 159s 3ms/step - loss: 1.4273 - accuracy: 0.5135 - val_loss: 1.2914 - val_accuracy: 0.6069
Epoch 4/20
50000/50000 [==============================] - 159s 3ms/step - loss: 1.1946 - accuracy: 0.5951 - val_loss: 1.1145 - val_accuracy: 0.6608
Epoch 5/20
50000/50000 [==============================] - 159s 3ms/step - loss: 1.0318 - accuracy: 0.6494 - val_loss: 0.9340 - val_accuracy: 0.6907
Epoch 6/20
50000/50000 [==============================] - 159s 3ms/step - loss: 0.9054 - accuracy: 0.6890 - val_loss: 0.8408 - val_accuracy: 0.7176
Epoch 7/20
50000/50000 [==============================] - 159s

In [10]:
score = net_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8807476462364197
Test accuracy: 0.7809000015258789


In [ ]:
#####
##### Benchmark 3 epoches: Normal and Data Gen
#####

In [ ]:
##### Normal Fit

In [9]:
history = net_final.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 171s 3ms/step - loss: 3.1071 - accuracy: 0.1970 - val_loss: 2.4319 - val_accuracy: 0.3496
Epoch 2/3
50000/50000 [==============================] - 157s 3ms/step - loss: 1.8556 - accuracy: 0.3754 - val_loss: 1.8706 - val_accuracy: 0.5086
Epoch 3/3
50000/50000 [==============================] - 157s 3ms/step - loss: 1.4487 - accuracy: 0.5082 - val_loss: 1.3071 - val_accuracy: 0.6050


In [10]:
score = net_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.307129285621643
Test accuracy: 0.6050000190734863


In [9]:
##### With Data Generator

In [10]:
##### Data Generator

train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_datagen.fit(x_train)

In [11]:
history_DG = net_final.fit_generator(train_datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                                     steps_per_epoch=len(x_train) / BATCH_SIZE, epochs=NUM_EPOCHS,
                                     validation_data=(x_test, y_test))

Epoch 1/3
1563/1562 [==============================] - 173s 111ms/step - loss: 2.7159 - accuracy: 0.1075 - val_loss: 12.6961 - val_accuracy: 0.1485
Epoch 2/3
1563/1562 [==============================] - 160s 102ms/step - loss: 2.4300 - accuracy: 0.1195 - val_loss: 9.7545 - val_accuracy: 0.1654
Epoch 3/3
1563/1562 [==============================] - 159s 102ms/step - loss: 2.3427 - accuracy: 0.1234 - val_loss: 5.1528 - val_accuracy: 0.1886


In [12]:
score = net_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 5.152817015075684
Test accuracy: 0.18860000371932983
